In [ ]:
from urllib.request import urlopen
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn as sns
from sklearn.mixture import GaussianMixture

In [ ]:
data = pd.read_csv(urlopen("https://raw.githubusercontent.com/cyndoro/CloudMonitoringDataset/main/Dataset.csv"))
data.head()

In [ ]:
print(data.shape)

In [ ]:
data.isnull().sum()

In [ ]:
data.describe().loc[['count','mean','min','max']]

In [ ]:
dataframe = pd.DataFrame(data)

In [ ]:
dataframe = dataframe.drop(["Microservices", "timestamp"], axis=1)
dataframe

In [ ]:
sns.pairplot(dataframe)

***Finding Optimal no.of Clusters***

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
Sumfsqrd_dis = []
K = range(1,10)
for num_clusters in K :
 kmeans = KMeans(n_clusters=num_clusters)
 kmeans.fit(dataframe)
 Sumfsqrd_dis.append(kmeans.inertia_)
plt.plot(K,Sumfsqrd_dis,'bx-')
plt.xlabel('Values of K') 
plt.ylabel('Sum of squared distances/Inertia') 
plt.title('Elbow Method For Optimal k')
plt.show()

***K-Means Clustering***

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2)
new = dataframe.filter(['CPU','mem','response','throughtput'], axis=1)
kmeans.fit(new)

Viewing centroids

In [ ]:
centroids = kmeans.cluster_centers_
Klabel = kmeans.labels_
print ("Cluster Centers are :", centroids)
print ("KLabels :", Klabel)
from collections import Counter
Counter(Klabel)

In [ ]:
Kdataframe = pd.concat([new, pd.DataFrame(Klabel, columns=["Cluster"])], axis=1)
#Kdataframe = DBdataframe.drop(new.columns[[3]],axis = 1)
Kdataframe

Plotting datapoints along with the centroids for Response vs Threshold Variables

In [ ]:
colours = ['m.','g.','r.', 'k.']
for i in range(len(new)):
    plot.plot(new.iloc[i][2], new.iloc[i][3], colours[Klabel[i]], markersize = 10 )
for i in range(len(centroids)):
    plot.plot(centroids[i][2], centroids[i][3], colours[i], markersize = 20, marker='*' )
plot.show()

In [ ]:
p = sns.scatterplot(data=Kdataframe, x="throughtput", y="response", hue=Kdataframe.Cluster, legend="full", palette="deep")
sns.move_legend(p, "upper right", bbox_to_anchor=(1.17, 1.2), title='Clusters')
plt.show()

***GMM***

In [ ]:
gmm = GaussianMixture(n_components = 2, init_params='random')
 
# Fit the GMM model for the dataset
# which expresses the dataset as a
# mixture of 3 Gaussian Distribution
gmm.fit(new)
cluster = gmm.predict(new) 
cluster
Counter(cluster)

#gmm.weights_
#gmm.means_
#gmm.covariances_

In [ ]:
Gdataframe = pd.concat([new, pd.DataFrame(cluster, columns=["Cluster"])], axis=1)
Gdataframe

In [ ]:
p = sns.scatterplot(data=Gdataframe, x="throughtput", y="response", hue=Gdataframe.Cluster, legend="full", palette="deep")
sns.move_legend(p, "upper right", bbox_to_anchor=(1.17, 1.2), title='Clusters')
plt.show()

**DBSCAN**

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=3, min_samples=13)
Dlabels = dbscan.fit_predict(new)
np.unique(Dlabels)
Counter(Dlabels)

In [ ]:
#DBdataframe = pd.concat([new, pd.DataFrame(Dlabels, columns=["Cluster"])], axis=1)
#DBdataframe = DBdataframe.drop(new.columns[[3]],axis = 1)
#df_filtered_noise = newdataframe[newdataframe.Cluster>-1]
#df_filtered_noise
#newdataframe

In [ ]:
# p = sns.scatterplot(data=df_filtered_noise, x="CPU", y="throughtput", hue=df_filtered_noise.Cluster, legend="full", palette="deep")
# sns.move_legend(p, "upper right", bbox_to_anchor=(1.17, 1.2), title='Clusters')
# plt.show()

Splitting Kmeans dataset into training(80%) and testing(20%)

In [ ]:
from sklearn.model_selection import train_test_split
data = Kdataframe[['CPU','mem','response','throughtput']]
X_train, X_test = train_test_split(Kdataframe, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

gaussianNB

In [ ]:
#importing gaussian naive bayes from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

#making an gaussian naive bayes model 
gnb = GaussianNB().fit(X_train[['CPU','mem','response','throughtput']],X_train['Cluster'])
gnb_predictions = gnb.predict(X_test[['CPU','mem','response','throughtput']])

#printing the accuracy of the generated model
gnbAccuracy = gnb.score(X_test[['CPU','mem','response','throughtput']],X_test['Cluster'])
print("Accuracy for Gaussian Naive Bayes: ",gnbAccuracy)

In [ ]:
#function for displaying the confusion matrix 
import seaborn as sns 
def draw_confusion_matrix(cm):
    plt.figure(figsize=(30,10))
    sns.heatmap(cm,annot=True,fmt="d", center=0, cmap='inferno') 
    plt.title("Confusion Matrix")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

#drawing a confusion matrix
cm = confusion_matrix(X_test['Cluster'], gnb_predictions)
draw_confusion_matrix(cm)

Support vector machine classifier

In [ ]:
#importing svm from sklearn
from sklearn import svm
from sklearn.metrics import confusion_matrix

#making an Svm model 
svm_model = svm.SVC(kernel = 'rbf').fit(X_resampled[['CPU','mem','response','throughtput']],X_resampled['Cluster'])
svm_predictions = svm_model.predict(X_test[['CPU','mem','response','throughtput']])

#printing the accuracy of the generated model
svmAccuracy = svm_model.score(X_test[['CPU','mem','response','throughtput']],X_test['Cluster'])
print("accuracy for svm: ",svmAccuracy)

#drawing a confusion matrix
cm = confusion_matrix(X_test['Cluster'], svm_predictions)
draw_confusion_matrix(cm)